In [2]:
a = [1,2,3,4,5,6,7,8]
a[a.index(1): a.index(5) + 1]

[1, 2, 3, 4, 5]

In [ ]:
import yaml
from datetime import datetime

# 读取YAML文件
with open(r'd:/workspace/TradeX/ezMoney/roll_back.yml', 'r') as file:
    configs = yaml.safe_load(file)

# 获取当前日期
current_date = datetime.now().strftime('%Y%m%d')

# 填充save_path字段
for config in configs['configs']:
    config['save_path'] = config['save_path'].format(strategy_name=config['strategy_name'], current_date=current_date)

# 打印更新后的配置
print(configs)
